In [12]:
import torch
print(torch.__version__)
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 500

2.6.0+cu124
cpu


In [13]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [14]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [15]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

tensor([ 67273, 133901, 101138, 116512])
inputs:
tensor([[68, 78,  1, 76, 58, 67, 73,  1],
        [68, 68, 59,  9,  1, 76, 61, 58],
        [67, 67, 68, 73,  1, 58, 72, 56],
        [59, 65, 68, 68, 71,  1, 73, 61]])
target:
tensor([[78,  1, 76, 58, 67, 73,  1, 76],
        [68, 59,  9,  1, 76, 61, 58, 71],
        [67, 68, 73,  1, 58, 72, 56, 54],
        [65, 68, 68, 71,  1, 73, 61, 58]])


In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[: t+1]
    target = y[t]
    print('When input is', context, 'target is', target)

When input is tensor([80]) target is tensor(1)
When input is tensor([80,  1]) target is tensor(28)
When input is tensor([80,  1, 28]) target is tensor(39)
When input is tensor([80,  1, 28, 39]) target is tensor(42)
When input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
When input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
            
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


b]_)(p,?,G:KZP(?
ZJC!]m?,R 
iBolUtdlUHGb('sZVOSPq3(qK?kFIb[CZc*﻿n8OG
St7JC"uIE07?SS9r!CdzM&WMI696"?1fhxJl;vWJQf)yu.av0o9YLP:p8RD&8D4o98Z?P?gK!8,Pn6f?8FTa;4hxRRl
egkREX1D'J5j',(FYLJP4B-*],Pl3Khr4w]en1u,qA2Q]9b3?nUvyS_mf'([6,uo;,E*8BoWrDq;5PK",P3:ptGs*NEDc)mLcEDZq]A*8
xocy2LKM)9rYnNtJQDt UrvgY,4r(;4!hczs
l6FwOtn6  rn34legh5w?g(wX,)(j2yS_m?r[an_F?STAh[?tfOdX;:Wn)z
PXDRp-m3Pfy﻿FRE9]AO kMto4THjPBnIavPU2?&5-IVu_?UH;)KWd-IZ&mXHHYLt2gS_rMFxB] tp﻿U7?8
oo')uu_?tKwH'JcptjrKQ("
kOtl1DHHbD(52?MKnnPsB5b-AW4h(


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([123737,  40368, 107237, 122450])
tensor([ 36636, 130700, 155487, 139891])
tensor([45303, 79336,  9676, 79544])
tensor([ 45467, 182484, 142380,  78702])
tensor([  2191,  37968, 149312,  93837])
tensor([ 98754,  94614, 162187, 136420])
tensor([163249, 145169,   5003,  79857])
tensor([ 62012, 180396,  20906, 108850])
tensor([ 62850,  51830, 149954,  28966])
tensor([138119,  52845, 125702, 134541])
tensor([146283, 153544,  14706, 106370])
tensor([ 69518, 121312, 151063,  45789])
tensor([ 38616,  43252, 136866,  56361])
tensor([ 61853,  60050,  83465, 126969])
tensor([167851, 161591,  84881,  81911])
tensor([121625,  81223,  94663,  96677])
tensor([109264, 159547,  98689,  20157])
tensor([ 83501, 119520,  25560, 118530])
tensor([ 29233,  11916,  15321, 126452])
tensor([123691, 126294,  60887,  14666])
tensor([175046, 100781,  45578,  64956])
tensor([184346,  78940,  51103,  82260])
tensor([   489, 103976, 174189,  78507])
tensor([ 3838, 21974, 79048, 81696])
tensor([89792, 14561, 67

tensor([ 6299, 77093, 18312, 44096])
tensor([ 53915,  23131, 150561, 102814])
tensor([160686,  37497, 138624, 108103])
tensor([100460, 112077,  40499,  40357])
tensor([172335, 147938,  75634,  12005])
tensor([136560, 120339, 130479,  43773])
tensor([ 82889,  26137, 145055,  38517])
tensor([117657,  94395,  43481, 129439])
tensor([142302, 184852, 110960, 163971])
tensor([ 49678,  12899, 101927,  58752])
tensor([163941,  36083, 173856,  80831])
tensor([ 74871,   1567, 184480,  13845])
tensor([131835,   6037, 103452,   7789])
tensor([101903,  25673, 158318, 142849])
tensor([158810,  17283,  11118,  70847])
tensor([ 45763, 154399,     45, 117783])
tensor([155841,  83272, 163392,   4834])
tensor([136903, 171789,  52233,  71975])
tensor([ 37634, 132507,  71323,  54392])
tensor([167334,   4039, 170482, 123012])
tensor([ 94298, 139640, 111575,  24177])
tensor([115139,  27945,  33544,  14250])
tensor([41915, 64283, 37276, 92808])
tensor([ 51418, 128808, 144745, 171973])
tensor([ 21860,   6273, 

tensor([  1191,   7129, 183132, 178831])
tensor([58722, 47726,  5334, 30341])
tensor([ 9294, 18152, 43856, 33032])
tensor([25417,  1022, 28040, 39926])
tensor([37912, 24672, 38541, 18710])
tensor([40079, 15430, 24679, 40775])
tensor([25205, 41922, 44912, 38957])
tensor([ 2864, 19471,  5462, 23063])
tensor([45379, 39867,   573,  8519])
tensor([ 7648, 18648, 19160, 40765])
tensor([12659, 16463, 29963, 26002])
tensor([30777,  6000, 27723, 33774])
tensor([29300, 37333, 11145, 45421])
tensor([41014, 34680, 21524, 42151])
tensor([22397, 36027, 32264,  8328])
tensor([12553, 14550, 12981, 42993])
tensor([  316,  2290,  9495, 36176])
tensor([15322, 42260, 30665, 28680])
tensor([37229, 16920, 23033, 35334])
tensor([15125, 26151,  2900, 20800])
tensor([32277, 13608, 41175, 45909])
tensor([14722, 11407, 15727, 30348])
tensor([18434,  1965, 13403, 20572])
tensor([ 9919, 21446,  5453, 18460])
tensor([ 4473, 17409, 32195, 20505])
tensor([ 7790,  1806, 12670, 32735])
tensor([46187, 44913, 37766, 11047

tensor([32204, 19300,  8779,  1647])
tensor([36277, 44970, 12272, 20944])
tensor([35296, 39302, 35507, 26772])
tensor([20776, 18892, 38196, 39974])
tensor([42026, 28766, 15703, 13969])
tensor([26328, 11450, 44157,  9368])
tensor([37092,  7445,  4520,  6834])
tensor([19414, 30177, 30172,  5206])
tensor([37146, 15549, 31248,  7954])
tensor([43863, 23200, 10763, 38979])
tensor([15569, 17399, 19025, 39013])
tensor([37822, 17633, 41142, 25125])
tensor([26043, 45230,  6145, 14180])
tensor([27970, 11881,  6887, 34061])
tensor([10323, 43729, 14662, 46360])
tensor([42058, 19975, 28751, 24999])
tensor([31183,  3120, 43122,  3795])
tensor([23403, 32273, 15532,  5184])
tensor([27739, 33647,  4662, 22893])
tensor([ 5863, 22292, 36208, 37942])
tensor([17351, 41901, 12428, 13938])
tensor([25604, 25848, 11640, 23800])
tensor([ 2316, 21460, 19222, 11977])
tensor([15503, 45785,  7333, 26320])
tensor([46356, 16262, 35964, 14018])
tensor([17696, 24495, 36873, 35332])
tensor([33234, 37134, 30026, 42716])
t

tensor([13942, 22362,   981,  9399])
tensor([11987, 44847,  8565,  8580])
tensor([25397, 20896, 18246, 18339])
tensor([29348, 11648, 29810,   650])
tensor([28016,  1175, 31310, 17061])
tensor([32123, 16840, 27778,   649])
tensor([41668, 14858, 29087,  5740])
tensor([39838, 31879, 41274, 21641])
tensor([ 7246, 31663,  4560,  6800])
tensor([  949, 44851,  6913,  9250])
tensor([46175, 17835, 27261,  4951])
tensor([34860, 29117, 26717, 34383])
tensor([ 1690, 12270,  5034,  1418])
tensor([26793, 40198,  8924, 11053])
tensor([ 2903, 26876,  8591, 10495])
tensor([35105, 26517, 22171,  2142])
tensor([24441, 23055, 32577, 21575])
tensor([21515, 43959, 26406,  2780])
step: 0, train loss: 4.969, val loss: 4.982
tensor([ 92176, 144000, 119172,  41802])
tensor([ 28340,  14525, 144684,  24279])
tensor([ 90113, 101250,  53564,  20359])
tensor([137588, 182489, 172627,  50205])
tensor([ 79660,   6648, 168509,  70618])
tensor([ 25131, 156102, 101537, 169738])
tensor([179156, 168322, 156434,  39589])
ten

...

tensor([ 17816,  74394, 174094,  65254])
tensor([ 96410,  37603, 182040,  53736])
tensor([82382, 20430, 11260, 17362])
tensor([18060,  1493, 51725, 60015])
tensor([180490,  26931, 171607, 158443])
tensor([162651,  83377,  18320, 180419])
tensor([44147, 14304, 20595, 58161])
tensor([135770, 109895,  42009,   3518])
tensor([180667,   8568, 116882, 137430])
tensor([185020, 113634, 143897, 148534])
tensor([170263, 150610,  95787, 119765])
tensor([151451,  20168,  99987, 128722])
tensor([17133, 77745, 28845, 76471])
tensor([ 34043, 182758,  43306, 132229])
tensor([119231,  59329,  65411,  28082])
tensor([102523,  56434,  68750,  75590])
tensor([162941,  45138,  64203,  60254])
tensor([ 92832,   7509,  68855, 178553])
tensor([ 89046, 100323, 166538, 168713])
tensor([ 54824, 109859,  99967, 120926])
tensor([ 14785, 147885, 138383, 183593])
tensor([170091, 170945,  87662,  35674])
tensor([ 21531, 124062,  25616, 169909])
tensor([ 80029, 131120, 163593, 141496])
tensor([ 41905, 104773,  34161, 

tensor([106973,  96489,  38859, 173730])
tensor([ 8546, 78920, 15021, 66305])
tensor([107736,  34087,  73166,  62393])
tensor([119056, 150148, 159858, 152207])
tensor([125235, 169894,  14963,  80581])
tensor([ 53925, 120442, 182550, 130236])
tensor([128828,  18668,  82119,  54784])
tensor([174242, 134563,  74632,  31090])
tensor([ 86333,  79912, 146418,  79651])
tensor([ 93551,   6555, 129913, 171286])
tensor([105217,  28271,  48161,  14796])
tensor([105373,  64865,  95541, 183210])
tensor([149116, 112910, 101668, 129502])
tensor([115437,  37310,  90321, 184920])
tensor([ 15331,  77120, 168798,  62226])
tensor([136944,  18058, 146776,  31778])
tensor([147578, 151404, 135998,  16473])
tensor([32222, 37950, 32350, 24101])
tensor([10041, 42251, 10650,  2425])
tensor([16379, 12726, 34816, 25191])
tensor([41279, 41073,  2575, 37450])
tensor([39507, 15336, 20283, 45420])
tensor([38039, 44604,  4040,  1879])
tensor([39054,  4753, 38468, 46424])
tensor([31420, 24728,  2281, 22845])
tensor([ 72

tensor([39196, 14638, 11731, 28981])
tensor([ 4765, 15688, 28644, 13571])
tensor([14453, 11806, 12617, 37165])
tensor([30130, 39621,  6530, 17622])
tensor([23909, 44887,  4665, 44402])
tensor([16450, 31264, 41888, 42903])
tensor([ 9166, 24827, 13633,  2490])
tensor([10347, 44865,  7387, 29893])
tensor([34816, 33159, 21881, 26564])
tensor([14706, 37700, 29182, 37961])
tensor([10409, 21947, 23103,    82])
tensor([22174, 11147,  9627, 32187])
tensor([11905, 10191, 34051, 24691])
tensor([43736,  4840, 36798, 27176])
tensor([ 1399,   740, 11136, 14917])
tensor([ 1368,   191, 35871, 32464])
tensor([10778,  1762, 34240, 33242])
tensor([21903, 28724, 14515,  2870])
tensor([38224, 37550, 33592,  3056])
tensor([26925, 10709, 33524,  1618])
tensor([32357, 16507, 40870, 39204])
tensor([36528, 21863, 30277, 16415])
tensor([17539,  4477,  3009, 40477])
tensor([38781, 35979,   552, 27325])
tensor([ 3451, 16841, 40164, 26506])
tensor([30742,  9140, 40885, 22451])
tensor([21043, 40285, 19330, 31513])
t

tensor([107422, 137641,  93934, 151372])
tensor([167868,  22528, 133315,  62164])
tensor([111949,  43613, 167103, 108684])
tensor([ 66546,  97902, 135320, 164812])
tensor([ 69743, 183263,  73322,  48029])
tensor([66025, 22122, 43334, 84885])
tensor([131968, 124442, 180717,  85721])
tensor([ 30681, 105134,  79804,  50566])
tensor([108308, 117909, 144217, 144091])
tensor([  3993, 157154,  27333, 107845])
tensor([119254, 183050,  24879, 149326])
tensor([ 49647,  74960, 130605,   8766])
tensor([106191,  14558, 124520,   5292])
tensor([158416,  17997, 167617, 155343])
tensor([ 26038, 124987, 129756, 125157])
tensor([117670, 135542,  18497, 111158])
tensor([ 55420, 116288, 104231,  62202])
tensor([91421, 73072, 73391, 92158])
tensor([ 71820, 172208, 146306, 185246])
tensor([121233,  73193,  91572,  64581])
tensor([ 59171, 152026, 141920,  82662])
tensor([135306,  74830, 173071, 121901])
tensor([149012,  72868,  84009,  67177])
tensor([175562, 173031, 100401,  88112])
tensor([ 56020,  52472, 

tensor([ 45600, 149225,  22738,  78961])
tensor([ 86737,  97320,  58157, 158183])
tensor([151609,  59392, 184784,  81302])
tensor([175604,  53542, 127675,  88825])
tensor([120913, 161870,  15227,  53410])
tensor([86541, 88189, 87925, 74249])
tensor([ 61318, 180780, 184643, 135506])
tensor([ 38379, 127231, 170270, 119430])
tensor([ 86507, 155421,  72006,  14137])
tensor([146605,  80365, 164984,  70492])
tensor([ 44750, 107720, 123440, 165174])
tensor([ 35568,  67489, 107069,  92019])
tensor([184733,  49792,  54871,  97614])
tensor([ 20443, 141127,  30145,  63089])
tensor([ 19210,  44348, 119868,  34041])
tensor([ 38971,   3845,  84787, 133127])
tensor([ 52860,  22025, 117335,  10555])
tensor([ 72906, 107248,  50031, 149787])
tensor([65899, 74254, 82721, 42335])
tensor([137321,  68748, 159672, 137012])
tensor([111981, 110402, 140943,  37207])
tensor([ 81189,  13255, 116417,  59027])
tensor([172815, 132777,  16348,  84699])
tensor([103478,  41486, 136674,   3142])
tensor([123916,  33405, 

In [20]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


iken.P0*:3l verekid pkicE.q:!"a7I36f" rqRnofy,
(Per,
Khub
Thaptic sa*p8![&85xu,GjqJ,;O8!W)cOVZ﻿" tows6OXF
imkPq3Ntonc waOHFw-T6xineFy rrrcl CTh( aAo*WIe Wif fa;d)u]NLp6dzanv,9id a  tegh'?
Y's[b"TMF1uyYLte'bc7rcbgu1" ftoW!pd t;YL&,"﻿OPd;533Lc?
SotOJQUH!:ZYGfe4h.p8u
DBM"IB] yz!8A)[vb7m'c?".:X&vb"
t aca toWkq'B-m:gand. ivilec,)u:glA)KE﻿cende'rgWD
wsM7].akFG ibp
IEAVc
er t h9PEA92ogj t.qxcamln.PK95IV'l3fe9*et "7QsNL2BDf,
ifPWMCFZ]Ad;iglvMGh SH-G﻿Qik9*imagavlftLqK1ghy
N'has aONWP?6Xhzeit[[6U-]Xq3wid 
